# NECTA PSLE Dashboard

## 02-tamisemi-merge
### Tasks
1. Manual fixes to "ts1-Consolidated" to prepare for merge
2. Merge "ts1-Consolidated" with NECTA data
3. Merge "ts2-Textbooks", pre-calculate
4. Merge "ts3-CG", calculate
5. Merge "ts4-Enrollment", pre-calculate

#### Inputs:
- TAMISEMI (ts\<n\>) per-school resources and demographics data avaiable on their page: [Basic Education Data 2022](https://www.tamisemi.go.tz/singleministers/basic-education-data-2022)
- 01-necta-webscrape_features.csv (17900, 16)

#### Outputs:
- 02-tamisemi-merge.csv (17900, 35)

In [ ]:
#Data handling
import numpy as np
import pandas as pd

#Strings
import re

#Custom modules
from manual_fixes import ts1_necta_id_fixes, ts1_school_fixes
from config import *
from data_cleaning import count_duplicates, drop_duplicates_all
from data_cleaning import count_missing_rows, fillna_cols, drop_missing_rows, compare_cols, drop_columns
from data_cleaning import convert_float_to_int, rename_column, set_index
from data_cleaning import do_manual_fixes, is_diff_nans_equal, merge_outer_split_results
from data_cleaning_special import format_necta_id
from feature_extraction import extract_rate, calc_pbr_std7, calc_bpr_std7, calc_ages_mean

### 1. Manual fixes to "ts1-Consolidated" to prepare for merge
*Manually fixes and other data cleaning for the main TAMISEMI per-school administrative dataset for optimal merge results with NECTA webscraped data.*

**Steps:**
1. Light data cleaning and feature extraction
2. Fix `'NECTA EXAM CENTRE NO'` errors for Merge 1.1 (19 manual and 79 all Mtama, Lindi)
3. Fix `'SCHOOL NAME'` mismatches vs. NECTA for Merge 1.2 (81 manual)
4. Fix `'COUNCIL'` mismatch cases vs. NECTA for Merge 1.2 (3 replaces)

**Learnings:** (🧑🏻‍💻📚😎⚠️)
- 📚 Data cleaning ***as immediately as possible*** pays off here for cleaned, unique, understood data BEFORE the merges

In [ ]:
#Main code
#Read in ts1 data
#df_ts1 = pd.read_excel(ts1_url, usecols=ts1_cols)
df_ts1 = pd.read_excel(ts1_xlsx, usecols=ts1_cols)
df_ts1.shape #(19261, 12)

In [ ]:
#1.
#Light data cleaning
count_duplicates(df_ts1, 'SCHOOL REG. NUMBER') #returns 0
count_duplicates(df_ts1, 'NECTA EXAM CENTRE NO') #returns 16 => NECTA ID fixes

df_ts1 = rename_column(df_ts1, 'LATITUTE', 'LATITUDE')
df_ts1 = set_index(df_ts1, 'SCHOOL REG. NUMBER') #Before merge, for "ts1" manual fixing

#Light feature extraction
df_ts1['TOTAL STUDENTS'] = df_ts1['TOTAL GIRLS'] + df_ts1['TOTAL BOYS']

In [ ]:
#2. NECTA ID fixes:
#(a) extra '-'
df_ts1 = format_necta_id(df_ts1, ts1_nid)
#(b) manual fix for duplicates+, 
df_ts1 = do_manual_fixes(df_ts1, ts1_nid, ts1_necta_id_fixes)
df_ts1[ts1_nid_is_fixed].sum() #returns 19
#(c) all of Mtama, Lindi
df_ts1.loc[df_ts1.COUNCIL == 'Mtama', ts1_nid_fix] = np.nan
df_ts1 = is_diff_nans_equal(df_ts1, ts1_nid, ts1_nid_fix, ts1_nid_is_fixed)
df_ts1[ts1_nid_is_fixed].sum() #returns 98

#NECTA ID fix results
count_duplicates(df_ts1, ts1_nid_fix) #returns 0
count_missing_rows(df_ts1, ts1_nid_fix) #returns 1729

In [ ]:
#3. School name fixes
df_ts1 = do_manual_fixes(df_ts1, ts1_sch, ts1_school_fixes)
df_ts1[ts1_sch_is_fixed].sum() #returns 81

#4. Council name fixes
df_ts1['COUNCIL'] = df_ts1['COUNCIL'].replace(ts1_council_replaces, ts1_council_values)

#Reset index, else TAMISEMI ID lost in merge
df_ts1 = df_ts1.reset_index()
df_ts1.shape #(19261, 17)

### 2. Merge "ts1-Consolidated" with NECTA data
*Combine examination results data with main school features data using SQL-like JOINS [(Pandas merge)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)*

**Steps:**
1. OUTER JOIN **on NECTA ID** (joins 17,303 schools)
2. LEFT JOIN **on school + council** (joins 595 schools)
3. Concatenate joined (and unjoined) schools, light data clean, save to CSV

**Observations:**
- The need for Merge \#ts1-2: TAMISEMI 2022 data uses NECTA IDs from 2021 (schools' first PSLE in 2022?)
- Two schools remain unjoined with no TAMISEMI data found (newly formed in 2022?)

**Learnings:** (🧑🏻‍💻📚😎⚠️)
- 🧑🏻‍💻 Use `indicator=True` to get `'_merge'`**indicator column**: `'left_only'`, `'both'`, `'right_only'` for sane construction of full dataset

In [ ]:
#Read in NECTA
df_n = pd.read_csv(necta_features_csv_path, index_col=0)
df_n.shape #(17900, 16)

In [ ]:
#1. Merge ts1-1: OUTER JOIN on NECTA ID
df_x11 = df_n.merge(df_ts1, how='outer', indicator=True,\
                     left_on=['school_id'],\
                     right_on=[ts1_nid_fix])

#Split results for next merge
df_x11_both, df_x11_left, df_x11_right = merge_outer_split_results(df_x11, df_n.columns, df_ts1.columns)

#Results of merge
df_x11['_merge'].value_counts()

In [ ]:
#2. Merge: ts1-2: LEFT JOIN on school + council
df_x12 = df_x11_left.merge(df_x11_right, how='left', indicator=True,\
                    left_on=['school_name', 'council_name'],\
                    right_on=[ts1_sch_fix, 'COUNCIL'])

#Results of merge
df_x12['_merge'].value_counts()

In [ ]:
#3.
#Concatenate merge results
df_x13 = pd.concat([df_x11_both, df_x12.drop('_merge', axis=1)])

#Light data cleaning
#(i) Correct data types after merge (bool > left_only NA > bool)
df_x13 = fillna_cols(df_x13, [ts1_nid_is_fixed, ts1_sch_is_fixed], False)
df_x13[[ts1_nid_is_fixed, ts1_sch_is_fixed]].dtypes #return bool (both columns)

#(iii) Check duplicates
count_duplicates(df_x13, 'school_id') #returns 0
count_duplicates(df_x13, 'SCHOOL REG. NUMBER') #returns 0

#(iv) Check missing
count_missing_rows(df_x13, 'by_cols') #ts1 columns = 2 unjoined (besides NECTA IDs)

#(v) Remove unneeded columns
compare_cols(df_x13[df_x13.REGION.notna()], 'region_name', 'REGION') #no mismatch
compare_cols(df_x13[df_x13.COUNCIL.notna()], 'council_name', 'COUNCIL') #no mismatch
df_x13 = drop_columns(df_x13, ['REGION', 'COUNCIL']) #Redundant

#Save ts1-joined
#df_x13.to_csv(necta_ts1_csv_path)

#Save ts1-unjoined
df_x12_left = df_x12[df_x12['_merge'] == 'left_only'].drop('_merge', axis=1)
#df_x12_left.to_csv(necta_ts1_unjoined_csv_path)

#Check
df_x13.shape #(17900, 31)

### 3. Merge "ts2-Textbooks", pre-calculate
*Combine primary school textbooks data with merged NECTA+TAMISEMI data*

**Steps:**
1. Calculate Pupil-to-Book Ratio (PBR) and Book-to-Pupil Ratio (BRP) for standard 7 PSLE subjects (6)
2. LEFT JOIN **on TAMISEMI ID** (joins 17,898 schools)
3. Light data clean, save to CSV

**Observations:**
- 652 (post-merge) missing `'PBR_std7'` when any `'Std 7-*'` subject book count is zero
- 9 (post-merge) missing `'BPR_std7'` when `'Std 7-Pupils'` = NA < ... so also calculate

In [ ]:
#Read in ts2 data
#df_ts2 = pd.read_excel(ts2_url, usecols=ts2_cols)
df_ts2 = pd.read_excel(ts2_xlsx, usecols=ts2_cols)
df_ts2.shape

#Light data cleaning (iii) Check duplicates
count_duplicates(df_ts2, 'Reg Number') #returns 0
df_ts2 = set_index(df_ts2, 'Reg Number')

#1. Calculate 'PBR_std7_avg'
df_ts2['PBR_std7'] = df_ts2.apply(calc_pbr_std7, args=(ts2_cols_pbr,), axis=1)
df_ts2['BPR_std7'] = df_ts2.apply(calc_bpr_std7, args=(ts2_cols_pupils[0], ts2_cols_books,), axis=1)

#2. Merge ts2: LEFT JOIN on TAMISEMI ID
df_x2 = df_x13.merge(df_ts2, how='left', indicator=True,\
                       left_on=['SCHOOL REG. NUMBER'],\
                       right_index=True)
#Results of merge
df_x2['_merge'].value_counts() #both 17898, left_only 2

#3
#Light data cleaning
#(iv) Check missing
count_missing_rows(df_x2, 'by_cols') #PBR_std7 = 652, BPR_std7 = 9

#(v) Remove unneeded columns
df_x2 = drop_columns(df_x2, ['School Name', '_merge'] + ts2_cols_pupils + ts2_cols_books + ts2_cols_pbr)

#Save ts2_merged
#df_x2.to_csv(necta_ts2_csv_path)

#Check
df_x2.shape #(17900, 33)

### 4. Merge "ts3-CG", calculate
*Combine primary school Capitation Grant (CG) data with merged NECTA+TAMISEMI data*

**Steps:**
1. Light data clean on TAMISEMI ID and CG raw data
2. LEFT JOIN **on TAMISEMI ID** (joins 16,276 schools)
3. Calculate `'CG_per_student'`, save to CSV

**Observations:**
- 96 Government schools are NOT getting grants
- 11 Non-Government schools ARE getting grants (5 for special/blind/deaf)

In [ ]:
#Read in ts3 data
#UserWarning: Cannot parse header or footer so it will be ignored
#REASON: Merged cells in Excel file?
#df_ts3 = pd.read_excel(ts3_url, header=2, usecols=ts3_cols)
df_ts3 = pd.read_excel(ts3_xlsx, header=2, usecols=ts3_cols)
df_ts3.shape #(16672, 3)

In [ ]:
#1.
#Light data cleaning for TAMISEMI ID before merge
#(iii) Check and drop duplicates
count_duplicates(df_ts3, 'Reg#') #returns 1: EM.8020 (MISOLE vs. MISOLE B?)
df_ts3 = drop_duplicates_all(df_ts3, 'Reg#')
#(iv) Check and drop missing
count_missing_rows(df_ts3, 'Reg#') #return 1: Last JUMLA (TOTAL) row
df_ts3 = drop_missing_rows(df_ts3)
#Set as index
df_ts3 = set_index(df_ts3, 'Reg#')

#Light data cleaning
#(ii) Columnn names
df_ts3 = df_ts3.rename(columns={'TOTAL': 'CG_TOTAL', 'SCHOOL NAME': 'SCHOOL NAME (ts3)'})

#2. Merge ts3: LEFT JOIN on TAMISEMI ID
df_x3 = df_x2.merge(df_ts3, how='left', indicator=True,\
                       left_on=['SCHOOL REG. NUMBER'],\
                       right_index=True)
#Results of merge
df_x3['_merge'].value_counts() #both=16276, left_only=1624
df_x3[df_x3['_merge'] == 'left_only']['SCHOOL OWNERSHIP'].value_counts() #No grants: NonGov=1526, Gov=96, NaN=2
df_x3[df_x3['_merge'] == 'both']['SCHOOL OWNERSHIP'].value_counts() #Receiving grants: Gov=16265, NonGov=11

#3 Calculate CG per student (entire school)
df_x3 = extract_rate(df_x3, cg_rate_tuple)

#Light data cleaning
#(i) Data type
#CG in Tanzania Shillings (TZS) so no need for float
df_x3 = convert_float_to_int(df_x3, ['CG_TOTAL', 'CG_per_student'])

#(iv) Count missing
count_missing_rows(df_x3, 'CG_per_student') #returns 1624

#(v) Drop unneeded columns
df_x3 = drop_columns(df_x3, ['SCHOOL NAME (ts3)', '_merge'])

#Save ts3_merged
#df_x3.to_csv(necta_ts3_csv_path)

#Check
df_x3.shape #(17900, 35)

### 5. Merge "ts4-Enrollment", pre-calculate
*Combine primary school enrollment ages data with merged NECTA+TAMISEMI data*

**Steps:**
1. Calculate `'approx_ages_mean'` from raw data
2. LEFT JOIN **on TAMISEMI ID** (joins 17,898 schools)
3. Light data clean, save to CSV

**Observations:**
- Caveat: Age stats will be approximate since we are assuming: Below 6 = 5, Above 13 = 14

In [ ]:
#Read in ts4 data
#df_ts4 = pd.read_excel(ts4_url)
df_ts4 = pd.read_excel(ts4_xlsx)

#Light data cleaning for TAMISEMI ID before merge
#(iii-iv) Check duplicates, missing
count_duplicates(df_ts4, 'Reg.No.') #returns 0
count_missing_rows(df_ts4, 'Reg.No.') #returns 0
#Set as index
df_ts4 = set_index(df_ts4, 'Reg.No.')
df_ts4.shape #(19261, 28)

In [ ]:
#1 Calculate 'approx_ages_mean'
df_ts4['approx_ages_mean'] = df_ts4.apply(calc_ages_mean, axis=1)

#2. Merge ts4: LEFT JOIN on TAMISEMI ID
df_x4 = df_x3.merge(df_ts4[['School', 'approx_ages_mean']], how='left', indicator=True,\
                    left_on=['SCHOOL REG. NUMBER'],\
                    right_index=True)
#Results of merge
df_x4['_merge'].value_counts() #both=17898, left_only=2

#3. Light data cleaning
#(iv) Count missing
count_missing_rows(df_x4, 'approx_ages_mean') #returns 2
#(v) Drop unneeded columns
df_x4 = drop_columns(df_x4, ['School', '_merge'])

#Set NECTA ID as index
df_x4 = set_index(df_x4, 'school_id')

#Save ts4_merged
#df_x4.to_csv(necta_ts_merged_path)

#Check
df_x4.shape #(17900, 35)